<a href="https://colab.research.google.com/github/arthurcaique/curso_verao_uninabuco/blob/master/sentiment_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
from string import punctuation

from pandas import read_csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import download as nltk_downloader
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

nltk_downloader('punkt')
nltk_downloader('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
eng_stop_words = stopwords.words('english')

In [0]:
def pre_process_text(text: str):
  text = text.lower()
  return ' '.join([token for token in word_tokenize(text) 
                   if token not in punctuation and token not in eng_stop_words]).strip()

In [103]:
train_df = read_csv('train.tsv', sep = '\t', encoding='utf-8')
print(train_df.shape)

train_df.drop_duplicates(subset='SentenceId', keep='first', inplace=True)
print(train_df.shape)

train_df['Phrase'] = train_df['Phrase'].apply(
    lambda phrase: pre_process_text(phrase))

train_y = train_df['Sentiment'].values

(156060, 4)
(8529, 4)


In [104]:
test_df = read_csv('test.tsv', sep = '\t', encoding='utf-8')
print(test_df.shape)

test_df.drop_duplicates(subset='SentenceId', keep='first', inplace=True)
print(test_df.shape)

test_df['Phrase'] = test_df['Phrase'].apply(
    lambda phrase: pre_process_text(phrase))

(66292, 3)
(3310, 3)


In [0]:
train_df['Phrase'] = train_df['Phrase'].apply(
    lambda phrase: pre_process_text(phrase))

train_y = train_df['Sentiment'].values

In [0]:
vectorizer = TfidfVectorizer()
tf_idf_matrix = vectorizer.fit_transform(train_df['Phrase'].values)

In [0]:
rf_classifier = RandomForestClassifier(n_estimators=100)

In [108]:
rf_classifier.fit(tf_idf_matrix, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [0]:
test_ = vectorizer.transform(test_df['Phrase'].values)

In [110]:
rf_classifier.predict(test_)

array([3, 3, 1, ..., 2, 1, 0])